## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import math

print(tf.__version__)

In [ ]:
DF_COLUMNS = ['r', 'h','V']
FEATURES = DF_COLUMNS[0:len(DF_COLUMNS) - 1]
LABEL = DF_COLUMNS[2]

In [ ]:
# Create a dataset containing random radius (r) and height (h) columns and V which is pi*r*r*h
# min minumum value of r and h
# max maximum value of r and h
# size size of the data set
def create_df(min, max, size):
    # create df with columns r and h containing random valuse
    df = pd.DataFrame( (max - min) * np.random.random(size=(size, 2)) + min, columns=list('rh'))
    df["V"] = math.pi * df.r ** 2 * df.h
    return df

In [ ]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))

In [ ]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [ ]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [ ]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [ ]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

In [ ]:
min_val = 0.5
max_val = 2

df_train = create_df(min_val, max_val, 10000)
df_valid = create_df(min_val, max_val, 200)
df_test = create_df(min_val, max_val, 200)

In [ ]:
df_train.describe()

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

OUTDIR = 'cylinder_volume_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.DNNRegressor(hidden_units = [3, 8, 5, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 1000));
print_rmse(model, df_valid)

In [ ]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

In [ ]:
print(df_test.V.ndim)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure()
ax = fig.gca(projection='3d')

# Make data.
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

print(X)

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_surface(df_test.r, df_test.h, df_test.V, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

plt.show()